# Filtering expanded dataset

## Core Imports

In [ ]:
# Generic Imports
import matplotlib.pyplot as plt
import pandas as pd

# File I/O
from pathlib import Path
import csv, json, openpyxl

# Typing and Subclassing
from typing import Iterable, Optional, Union
from dataclasses import dataclass, field

# Cheminformatics
from rdkit import Chem

# Custom imports
from polymerist.rdutils import rdkdraw
from polymerist.rdutils.smileslib import queries
from polymerist.monomers import specification

DIM    = 300
ASPECT = 3/2
rdkdraw.set_rdkdraw_size(DIM, ASPECT)
rdkdraw.disable_substruct_highlights()

# Static Paths
RAW_DATA_DIR  = Path('monomer_data_raw')
FMT_DATA_DIR  = Path('monomer_data_formatted')
PROC_DATA_DIR = Path('monomer_data_processed')
RXN_FILES_DIR = Path('poly_rxns')
# RXN_FILES_DIR = Path('rxn_smarts')

## Loading desired dataset

In [ ]:
# input_data_path = FMT_DATA_DIR / 'nipu_urethanes_FMT.csv'
input_data_path = FMT_DATA_DIR / '20231114_polyid_data_density_DP2-6 - 1,2 monomers.csv'
# input_data_path = FMT_DATA_DIR / '221010_trainingdata_DP-18_expanded_FMT.csv'
df = pd.read_csv(input_data_path)

## Defining queries for illegal chemistries

In [ ]:
illegal_chem_queries = {
    'silicon' : Chem.MolFromSmarts('[Si]'),
    'sulfur'  : Chem.MolFromSmarts('[S]'),
    'metal'   : queries.SPECIAL_QUERY_MOLS['metal'],
    # 'halogen' : queries.SPECIAL_QUERY_MOLS['halogen'],
}

## Loading and backmapping premade reaction functional groups and templates

In [ ]:
rxn_backmap = { # map NREL dataset mechanism names to pre-made rxn template names
    'amide'     : 'polyamide',
    'carbonate' : 'polycarbonate_phosgene',
    'ester'     : 'polyester',
    'imide'     : 'polyimide',
    'urethane'  : 'polyurethane_isocyanate',
    'NIPU'      : 'polyurethane_nonisocyanate',
    'vinyl'     : 'polyvinyl_head_tail'
}
assert(set(df['mechanism'].unique()).issubset(set(rxn_backmap.keys()))) # verify that we've mapped all reactions

with (RXN_FILES_DIR / 'rxn_backmap.json').open('w') as backmap_file:
    json.dump(rxn_backmap, backmap_file, indent=4)

In [ ]:
with (RXN_FILES_DIR / 'fn_group_smarts.json').open('r') as file:
    fn_group_smarts  = json.load(file)
    
fn_group_queries = { # RDKit Mols generated from SMARTS queries
    group_name : Chem.MolFromSmarts(smarts)
        for group_name, smarts in fn_group_smarts.items()
}

with (RXN_FILES_DIR / 'rxn_groups.json').open('r') as file: # load table of functional group for each reaction
    rxn_groups = json.load(file)

In [ ]:
banned_monomers = [ # monomers which are, for one reason or another, disallowed
    'CC(C)(C)c1cc(c(Oc2ccc(cc2)N(c3ccc(N)cc3)c4ccc(N)cc4)c(c1)C(C)(C)C)C(C)(C)C'  # the extraordinary number of symmetries of this amine ("4-N-(4-aminophenyl)-4-N-[4-(2,4,6-tritert-butylphenoxy)phenyl]benzene-1,4-diamine")... 
]                                                                                 # ...mean it takes impractically long to isomorphism match during the Topology partition step

banned_monomer_queries = {}
for smiles in banned_monomers:
    exp_spi = specification.expanded_SMILES(smiles, assign_map_nums=False)
    banned_monomer_queries[smiles] = Chem.MolFromSmiles(exp_spi, sanitize=False)

## Utility functions

In [ ]:
from enum import Enum, auto

@dataclass
class MonomerInfo:
    '''For encapsulating salient info about an individual monomer'''
    position      : int
    smiles        : str
    fn_group_name : str
    IUPAC_name    : str

class FilterRejectionReason(Enum):
    '''For capturing information about which filtering step a monomer didn't pass'''
    BAD_NUM_MONOMERS      = auto()
    ILLEGAL_ATOMS         = auto()
    NO_RXN_TEMPLATE_MATCH = auto()
    BAD_NUM_FN_GRPS       = auto()
    NO_IUPAC_NAME         = auto()
    BANNED_MONOMER        = auto()

In [ ]:
import cirpy
import pubchempy as pcp

def get_IUPAC_name(smiles : str, pc_prop_name : str='IUPACName') -> Optional[str]:
    '''Takes the SMILES string representing a molecule and attempts to fetch its IUPAC name from NIH CACTUS and/or PubChem
    Returns the fetched IUPAC name as a str, or NoneType if both queries fail'''
    # Open with NIH query (fastest method), return name if found...
    iupac_name = cirpy.resolve(smiles, 'iupac_name')
    if iupac_name is not None:
        return iupac_name 
    
    # ...otherwise, search through PubChem Compound queries for a matching results
    for prop_query in pcp.get_properties(pc_prop_name, smiles, namespace='smiles'):
        if pc_prop_name in prop_query:
            return prop_query[pc_prop_name]
    else:
        return None # TODO : add ChemSpider once I can obtain an API key (https://chemspipy.readthedocs.io/en/latest/guide/intro.html#apikey) 

In [ ]:
from polymerist.monomers import specification
from polymerist.maths.combinatorics.sequences import bin_ids_forming_sequence

def process_monomer_input(smiles : str, mech_name : str) -> Union[FilterRejectionReason, list[MonomerInfo]]:
    '''For screening and expanding SMILES input to the MD polymer building workflow'''
    # 1) Determine number of distrinct monomers in passed SMILES string
    indiv_smiles = smiles.split('.') # opting for direct string-based approach here rather than RDKit Mol fragmenting for speed, and to avoid clunky str -> Mol -> str conversion
    num_monomers = len(indiv_smiles)
    if num_monomers != 2:
        return FilterRejectionReason.BAD_NUM_MONOMERS
    
    # Generate expanded SMILES string and corresponding RDKit Mol for all monomers
    exp_smiles         : list[str     ] = []
    indiv_monomers     : list[Chem.Mol] = []
    mono_group_choices : list[Iterable[str]] = []
    
    for smi in indiv_smiles: # wait to expand SMILES until after count filtering for speed
        exp_smi = specification.expanded_SMILES(smi, assign_map_nums=False)
        monomer = Chem.MolFromSmiles(exp_smi, sanitize=False)

        # 2) Catch monomers with illegal atoms or, if none are present, generate expanded SMILES string and corresponding RDKit Mol for all monomers
        if any(queries.matching_labels_from_substruct_dict(monomer, illegal_chem_queries)): # if any illegal atoms are detected in the current monomer, return and exit
            return FilterRejectionReason.ILLEGAL_ATOMS

        exp_smiles.append(exp_smi)
        indiv_monomers.append(monomer)
        mono_group_choices.append(queries.matching_labels_from_substruct_dict(monomer, fn_group_queries)) # generate ordered list of functional group choice bins

    # 3) Determine if monomer functionalizations match the advertised rxn, and if so in what order they should appear
    reactive_groups : list[str] = rxn_groups[rxn_backmap[mech_name]]
    all_indices     : set[int]  = set(range(num_monomers))
    for mono_order in bin_ids_forming_sequence(sequence=reactive_groups, choice_bins=mono_group_choices):
        if set(mono_order) == all_indices: # check to avoid duplication for multifunctional monomers
            break # exit loop when first valid order is found
    else:
        return FilterRejectionReason.NO_RXN_TEMPLATE_MATCH
    
    # produce monomers based on a valid ordering if one is found
    mono_info_list : list[MonomerInfo] = []
    for i, mono_fn_group in zip(mono_order, reactive_groups, strict=True): # strict just gives one further safeguard that a 1:1 reactive group to monomer mapping has been found
        monomer = indiv_monomers[i]
        # 4) Exclude any monomers which are manually disallowed
        if any(queries.matching_labels_from_substruct_dict(monomer, banned_monomer_queries)): # if any illegal atoms are detected in the current monomer, return and exit
            return FilterRejectionReason.BANNED_MONOMER

        # 5) Determine if all monomers are only have the desired functionalization numbers 
        if queries.num_substruct_queries_distinct(monomer, fn_group_queries[mono_fn_group]) != 2:
            return FilterRejectionReason.BAD_NUM_FN_GRPS
        
        # 6) Query IUPAC names for each monomer - NOTE: not done in the monomer expansion loop as the name query is the slowest step and should preferably alled as few times as possible
        iupac_name = get_IUPAC_name(exp_smiles[i])
        if iupac_name is None:
            return FilterRejectionReason.NO_IUPAC_NAME
        
        # if a monomer has gotten here, that means it's passed!
        mono_info = MonomerInfo(
            position=i,
            smiles=exp_smiles[i],
            fn_group_name=mono_fn_group,
            IUPAC_name=iupac_name
        )
        mono_info_list.append(mono_info)

    return mono_info_list

# Filter dataset

## Generate filtered Series (either error codes or MonomerInfo) from initial dataset

In [ ]:
from rich.progress import track

targ_df = df
# targ_df = df.head(30)
# targ_df = df[df.mechanism == 'vinyl']

proc_output = []
for (i, row) in track(targ_df.iterrows(), total=len(targ_df), description='Processing SMILES dataset...'):
    proc = process_monomer_input(row.smiles_monomer, row.mechanism)
    proc_output.append(proc)

proc_output = pd.Series(proc_output) # convert from list to Series, partition by failure state
was_rejected = proc_output.map(lambda out : isinstance(out, FilterRejectionReason))
rejects = proc_output[ was_rejected]
passes  = proc_output[~was_rejected]

## Expand successful monomers into labelled DataFrame

In [ ]:
def expand_mono_info_list(all_info : list[MonomerInfo]) -> list[str]:
    return {
        f'{attr}_monomer_{j}' : getattr(mono_info, attr)
            for j, mono_info in enumerate(all_info) # NOTE: the index "j" here is NOT the position as determined by monomer sort, but rather the index in the rxn groups ordering
                for attr in ('smiles', 'fn_group_name', 'IUPAC_name')
    }

passes_as_dict = passes.map(expand_mono_info_list)
passes_df = pd.DataFrame.from_records(passes_as_dict.values, index=passes_as_dict.index)

result_df = pd.concat([passes_df, df.loc[passes_df.index]], axis=1) # combine expanded monomer info with original dataframe
result_df.insert( # insert row of backmapped rxn name to aoid lookup in subsequent pipeline steps
    loc=result_df.columns.get_loc('mechanism'),
    column='rxn_name',
    value=result_df['mechanism'].map(rxn_backmap)
)

result_df

## Show distribution of failure cases

In [ ]:
dim = 6
aspect= 3/2

plt.figure(figsize=(dim*aspect, dim))

ax = rejects.map(lambda x : x.value).plot(kind='hist')
ax.set_xticks([i.value for i in FilterRejectionReason])
ax.set_xticklabels([i.name for i in FilterRejectionReason], rotation=-30)

plt.show()

## Save filtered DataFrame for next steps to avoid reprocessing

In [ ]:
PROC_DATA_DIR.mkdir(exist_ok=True)
clean_input_stem = input_data_path.stem.removesuffix('_FMT')

# save filtered dataset to filtered directory to store caclulations
out_data_path = PROC_DATA_DIR / f'{clean_input_stem}_FILTERED{input_data_path.suffix}'
result_df.to_csv(out_data_path)

# also keep record of the rejected monomers (and the reasons for rejection)
rejects_data_path = PROC_DATA_DIR / f'{clean_input_stem}_REJECTED{input_data_path.suffix}'
rejects.map(lambda r : r.name).to_csv(rejects_data_path)